###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

###### Translated by Huizhe Di @ SCSIO

# 敏感核计算通过二维声学有限差分建模


除了地震勘探建模之外，我们的二维声学有限差分代码还可以作为地震全波形反演(FWI)方法的核心。 一个非常有效的实现在频率域是可能的。

声学频率域的FWI目标是使数据残差最小化 $\mathbf{\delta \tilde{P} = \tilde{P}^{mod} - \tilde{P}^{obs}}$ 在建模的频率域数据 $\mathbf{\tilde{P}^{mod}}$ 和场数据 $\mathbf{\tilde{P}^{obs}}$ 为了推导出地下P波速度分布的高分辨率模型。 为了求解这个非线性反演问题，一个目标函数E，用来测量数据的不适合程度，被定义了出来。经典的选择是数据残差的L2范数

\begin{equation} 
E  = ||\mathbf{\delta \tilde{P}}||_2 = \frac{1}{2}\mathbf{\delta \tilde{P}}^\dagger \mathbf{\delta \tilde{P}} = \frac{1}{2} \sum_{k=1}^{n_\omega} \sum_{i=1}^{ns} \sum_{j=1}^{nr} \delta \tilde{P}^*(\mathbf{x_s}_i, \mathbf{x_r}_j, \omega_k) \delta \tilde{P}(\mathbf{x_s}_i, \mathbf{x_r}_j, \omega_k) \notag
\end{equation}  

这里 ns 和 nr 表示炮点和检波点的数量, $n_\omega$ 为离散频率的数量, $\dagger$ 为复杂的转置, $*$ 为共轭复数, $\mathbf{x_s},\; \mathbf{x_r}$ 分别为炮点和检波点的位置。通过迭代更新P波速度可以使目标函数最小化 $\mathbf{Vp}$ 在迭代步长 n, 从初始背景模型开始 $\mathbf{Vp_0}$, 沿搜索方向使用 **Newton** 方法:

\begin{equation} 
\mathbf{Vp}_{n+1} = \mathbf{Vp}_{n} - \mu_n \mathbf{H}_n^{-1} \left(\mathbf{\frac{\partial E}{\partial Vp}}\right)_n, \notag
\end{equation}  

这里 $\mu$ 表示步长, $\mathbf{\frac{\partial E}{\partial Vp}}$ 为梯度， ${\mathbf H}$ 为目标函数的二阶导数 (Hessian) 相较于 $\mathbf{Vp}$. 步长 $\mu_{n}$ 可以用不精确的抛物线搜索来估计.

梯度 $\mathbf{\frac{\partial E}{\partial Vp}}$ 可以通过下面的式子计算:

\begin{equation} 
\mathbf{\frac{\partial E}{\partial Vp}} = - \mathbf{K}^\dagger \delta P, \notag
\end{equation}  

这里的 $\mathbf{K}$ 代表 **敏感核**:

\begin{equation} 
K(x,z,\omega) = {\cal{Re}}\biggl\{\frac{2 \omega^2}{Vp^3} \frac{\tilde{P}(x,z,\omega,\mathbf{x_{s}}) \tilde{G}(x,z,\omega,\mathbf{x_{r}})}{max(\tilde{P}(x,z,\omega,\mathbf{x_{s}}))}\biggr\}
\end{equation} 

用单色前向波场 $\tilde{P}(x,z,\omega,\mathbf{x_s})$ 在源位置被激发 $\mathbf{x_s}$ 和格林函数 $\tilde{G}(x,z,\omega,\mathbf{x_r})$ 在接收器位置被激发 $\mathbf{x_r}$, $\cal{Re}$ 表示真实部分.

## 用离散傅里叶变换 (DFT) 从时间域波场计算单色频域域波场


为了计算等式(1)中的敏感核，我们首先需要从时间域波场估计单色频率域波场。这可以很容易地在我们的二维声学有限差分代码中实现通过 **离散傅里叶变换 (DFT)** 在有限差分代码的时间循环中。 我们近似于连续傅里叶变换

\begin{equation}
\tilde{f}(\omega) = \frac{1}{2 \pi} \int_{-\infty}^{\infty} f(t) exp(-i \omega t) dt \notag
\end{equation}

通过

\begin{equation}
\tilde{f_i}(\omega) \approx \frac{1}{2 \pi} \sum_{n=0}^{nt} f_n(t_n) \biggl(cos(\omega t_n)-i\; sin(\omega t_n)\biggr) dt \notag
\end{equation}

这里 $nt$ 表示有限差分代码中时间步长的数量, $\omega = 2 \pi f$ 圆频率基于频率 $f$, $i^2 = -1$. 波场可以进一步分解为实数 

\begin{equation}
Re\{\tilde{f_i}(\omega)\} \approx \frac{1}{2 \pi} \sum_{n=0}^{nt} f_n(t_n) \biggl(cos(2 \pi f t_n)\biggr) dt \notag
\end{equation}

和虚数部分

\begin{equation}
Im\{\tilde{f_i}(\omega)\} \approx -\frac{1}{2 \pi} \sum_{n=0}^{nt} f_n(t_n) \biggl(sin(2 \pi f t_n)\biggr) dt \notag
\end{equation}


有限差分代码中的实现非常简单。在时间步进过程中，我们必须将时域压力波场乘以一个几何因子，并将其贡献相加。让我们将其实现到我们的2D声学FD代码中，并尝试计算均匀背景模型的频域波场和Marmousi-2模型的初至波走时层析成像结果 ...

In [ ]:
# Import Libraries 
# ----------------
import numpy as np
from numba import jit
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams

# Ignore Warning Messages
# -----------------------
import warnings
warnings.filterwarnings("ignore")

from mpl_toolkits.axes_grid1 import make_axes_locatable

和往常一样，我们从基本建模参数的定义开始 ...

In [ ]:
# Definition of modelling parameters
# ----------------------------------

# Define model discretization
nx = 500    # number of grid points in x-direction
nz = 174    # number of grid points in z-direction
dx = 20.0   # spatial grid point distance in x-direction (m)
dz = dx     # spatial grid point distance in z-direction (m)

# Define xmax, zmax
xmax = nx * dx
zmax = nz * dz

# Define maximum recording time
tmax =  6.0 # maximum wave propagation time (s)

# Define source and receiver position
xsrc =  2000.0 # x-source position (m)
zsrc =  40.0   # z-source position (m)

xrec =  8000.0 # x-receiver position (m)
zrec =  40.0   # z-source position (m)

f0   = 10 # dominant frequency of the source (Hz)
print("f0 = ", f0, " Hz")
t0   = 4.0/f0   # source time shift (s)

isnap = 2  # snapshot interval (timesteps)

# Calculate monochromatic wavefields for discrete frequency freq
freq = 5.0 # discrete frequency (Hz)

... define a JIT-ed function for the spatial FD approximation ...

In [ ]:
@jit(nopython=True) # use JIT for C-performance
def update_d2px_d2pz_3pt(p, dx, dz, nx, nz, d2px, d2pz):
    
    for i in range(1, nx - 1):
        for j in range(1, nz - 1):
                
            d2px[i,j] = (p[i + 1,j] - 2 * p[i,j] + p[i - 1,j]) / dx**2                
            d2pz[i,j] = (p[i,j + 1] - 2 * p[i,j] + p[i,j - 1]) / dz**2
        
    return d2px, d2pz 

... 初始化吸收边界框架 ...

In [ ]:
# Define simple absorbing boundary frame based on wavefield damping 
# according to Cerjan et al., 1985, Geophysics, 50, 705-708
def absorb(nx,nz):

    FW = 60     # thickness of absorbing frame (gridpoints)    
    a = 0.0053
    
    coeff = np.zeros(FW)
    
    # define coefficients in absorbing frame
    for i in range(FW):    
        coeff[i] = np.exp(-(a**2 * (FW-i)**2))

    # initialize array of absorbing coefficients
    absorb_coeff = np.ones((nx,nz))

    # compute coefficients for left grid boundaries (x-direction)
    zb=0 
    for i in range(FW):
        ze = nz - i - 1
        for j in range(zb,ze):
            absorb_coeff[i,j] = coeff[i]

    # compute coefficients for right grid boundaries (x-direction)        
    zb=0
    for i in range(FW):
        ii = nx - i - 1
        ze = nz - i - 1
        for j in range(zb,ze):
            absorb_coeff[ii,j] = coeff[i]

    # compute coefficients for bottom grid boundaries (z-direction)        
    xb=0 
    for j in range(FW):
        jj = nz - j - 1
        xb = j
        xe = nx - j
        for i in range(xb,xe):
            absorb_coeff[i,jj] = coeff[j]

    return absorb_coeff

在二维有限差分声学建模代码中，我们实现了时间域波场的离散傅里叶变换，通过初始化压力波场的实部和虚部，计算时间循环内离散傅里叶变换的三角因子，对离散频率 `freq` 的压力波场 `p` 应用离散傅里叶变换，最终返回频率域波场的实部和虚部。

In [ ]:
# FD_2D_acoustic code with JIT optimization
# -----------------------------------------
def FD_2D_acoustic_JIT(vp,dt,dx,dz,f0,xsrc,zsrc,op,freq):        
    
    # calculate number of time steps nt 
    # ---------------------------------
    nt = (int)(tmax/dt)
    
    # locate source on Cartesian FD grid
    # ----------------------------------
    isrc = (int)(xsrc/dx)  # source location in grid in x-direction
    jsrc = (int)(zsrc/dz)  # source location in grid in x-direction    
    
    # Source time function (Gaussian)
    # -------------------------------
    src  = np.zeros(nt + 1)
    time = np.linspace(0 * dt, nt * dt, nt)

    # 1st derivative of Gaussian
    src  = -2. * (time - t0) * (f0 ** 2) * (np.exp(- (f0 ** 2) * (time - t0) ** 2)) 
    
    # define clip value: 0.1 * absolute maximum value of source wavelet
    clip = 0.1 * max([np.abs(src.min()), np.abs(src.max())]) / (dx*dz) * dt**2
    
    # Define absorbing boundary frame
    # -------------------------------    
    absorb_coeff = absorb(nx,nz)
    
    # Define squared vp-model
    # -----------------------        
    vp2 = vp**2
    
    # Initialize empty pressure arrays
    # --------------------------------
    p    = np.zeros((nx,nz)) # p at time n (now)
    pold = np.zeros((nx,nz)) # p at time n-1 (past)
    pnew = np.zeros((nx,nz)) # p at time n+1 (present)
    d2px = np.zeros((nx,nz)) # 2nd spatial x-derivative of p
    d2pz = np.zeros((nx,nz)) # 2nd spatial z-derivative of p 
    
    # INITIALIZE ARRAYS FOR REAL AND IMAGINARY PARTS OF MONOCHROMATIC WAVEFIELDS HERE! 
    # --------------------------------------------------------------------------------
    p_real = np.zeros((nx,nz)) # real part of the monochromatic wavefield 
    p_imag = np.zeros((nx,nz)) # imaginary part of the monochromatic wavefield 
        
    # Initalize animation of pressure wavefield 
    # -----------------------------------------    
    fig = plt.figure(figsize=(7,3))  # define figure size
    extent = [0.0,xmax,zmax,0.0]     # define model extension
    
    # Plot Vp-model
    image = plt.imshow((vp.T)/1000, cmap=plt.cm.gray, interpolation='nearest', 
                        extent=extent)    
    
    # Plot pressure wavefield movie
    image1 = plt.imshow(p.T, animated=True, cmap="RdBu", alpha=.75, extent=extent, 
                          interpolation='nearest', vmin=-clip, vmax=clip)    
    plt.title('Pressure wavefield')
    plt.xlabel('x [m]')
    plt.ylabel('z [m]')
           
    plt.ion()    
    plt.show(block=False)
    
    # Calculate Partial Derivatives
    # -----------------------------
    for it in range(nt):
    
        # FD approximation of spatial derivative by 3 point operator
        if(op==3):
            d2px, d2pz = update_d2px_d2pz_3pt(p, dx, dz, nx, nz, d2px, d2pz)

        # Time Extrapolation
        # ------------------
        pnew = 2 * p - pold + vp2 * dt**2 * (d2px + d2pz)

        # Add Source Term at isrc
        # -----------------------
        # Absolute pressure w.r.t analytical solution
        pnew[isrc,jsrc] = pnew[isrc,jsrc] + src[it] / (dx * dz) * dt ** 2
        
        # Apply absorbing boundary frame
        # ------------------------------
        p *= absorb_coeff
        pnew *= absorb_coeff
        
        # Remap Time Levels
        # -----------------
        pold, p = p, pnew
        
        # Calculate frequency domain wavefield at discrete frequency freq by DFT
        # ----------------------------------------------------------------------
        time = it * dt  # time
        trig1 = np.cos(2.0 * time * freq * np.pi) * dt # real part
        trig2 = np.sin(2.0 * time * freq * np.pi) * dt # imaginary part
        
        # Estimate real and imaginary part of pressur wavefield p by DFT
        # --------------------------------------------------------------
        p_real += p * trig1
        p_imag += p * trig2
    
        # display pressure snapshots 
        if (it % isnap) == 0:            
            image1.set_data(p.T)
            fig.canvas.draw()
    
    # Finalize computation of DFT
    p_real =   p_real / (2.0 * np.pi)
    p_imag = - p_imag / (2.0 * np.pi)
    
    # Return real and imaginary parts of the monochromatic wavefield
    return p_real, p_imag

## 均匀声学介质的单色频率域波场建模


现在，所有的东西都组装起来计算频域波场。我们只需要定义离散频率' freq '来计算单色波场。我们从均匀模型开始:

In [ ]:
# Run 2D acoustic FD modelling with 3-point spatial operater
# ----------------------------------------------------------
%matplotlib notebook
op = 3  # define spatial FD operator (3-point) 

# define homogeneous model with vp = 2500 m/s
vp_hom = 2500.0 * np.ones((nx,nz))

# Define time step
dt =  dx / (np.sqrt(2) * np.max(vp_hom))# time step (s)

p_hom_re, p_hom_im = FD_2D_acoustic_JIT(vp_hom,dt,dx,dz,f0,xsrc,zsrc,op,freq)

时间域波场似乎是正确的。让我们看一下频率域波场:

In [ ]:
%matplotlib notebook
# Plot real and imaginary parts of monochromatic wavefields
clip_seis = 5e-10
extent_seis = [0.0,xmax/1000,zmax/1000,0.0]

ax = plt.subplot(211)
plt.imshow(p_hom_re.T, cmap=plt.cm.RdBu, aspect=1, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)

plt.title('Real part of monochromatic wavefield')
#plt.xlabel('x [km]')
ax.set_xticks([]) 
plt.ylabel('z [km]')


plt.subplot(212)
plt.imshow(p_hom_im.T, cmap=plt.cm.RdBu, aspect=1, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)
plt.title('Imaginary part of monochromatic wavefield')
plt.xlabel('x [km]')
plt.ylabel('z [km]')
plt.tight_layout()
plt.show()

## 单色频率域波场建模对于Marmousi-2 走时层析成像模型

下面, 我们计算单色波场对于Marmousi-2 model的初至波走时层析成像模型, 它可以作为下一步FWI的初始速度模型。首先，我们导入这个模型:

In [ ]:
# Import FATT result for Marmousi-2 Vp model
# ------------------------------------------

# Define model filename
name_vp = "marmousi-2/marmousi_II_fatt.vp"

# Open file and write binary data to vp
f = open(name_vp)
data_type = np.dtype ('float32').newbyteorder ('<')
vp_fatt = np.fromfile (f, dtype=data_type)

# Reshape (1 x nx*nz) vector to (nx x nz) matrix 
vp_fatt = vp_fatt.reshape(nx,nz)

In [ ]:
# Plot Marmousi-2 vp-model
# ------------------------
%matplotlib notebook
extent = [0, xmax/1000, zmax/1000, 0]
fig = plt.figure(figsize=(7,3))  # define figure size
image = plt.imshow((vp_fatt.T)/1000, cmap=plt.cm.viridis, interpolation='nearest', 
                   extent=extent)


cbar = plt.colorbar(aspect=12, pad=0.02)
cbar.set_label('Vp [km/s]', labelpad=10)
plt.title('Marmousi-2 FATT model')
plt.xlabel('x [km]')
plt.ylabel('z [km]')
plt.show()

... 运行时间域的代码:

In [ ]:
# Run 2D acoustic FD modelling with 3-point spatial operater
# ----------------------------------------------------------
%matplotlib notebook
op = 3  # define spatial FD operator (3-point) 

# Define time step by CFL criterion
dt =  dx / (np.sqrt(2) * np.max(vp_fatt))# time step (s)

p_fatt_re, p_fatt_im = FD_2D_acoustic_JIT(vp_fatt,dt,dx,dz,f0,xsrc,zsrc,op,freq)

In [ ]:
%matplotlib notebook
# Plot real and imaginary parts of monochromatic wavefields
clip_seis = 5e-9
extent_seis = [0.0,xmax/1000,zmax/1000,0.0]

ax = plt.subplot(211)
plt.imshow(p_fatt_re.T, cmap=plt.cm.RdBu, aspect=1, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)

plt.title('Real part of monochromatic wavefield')
#plt.xlabel('x [km]')
ax.set_xticks([]) 
plt.ylabel('z [km]')


plt.subplot(212)
plt.imshow(p_fatt_im.T, cmap=plt.cm.RdBu, aspect=1, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)
plt.title('Imaginary part of monochromatic wavefield')
plt.xlabel('x [km]')
plt.ylabel('z [km]')
plt.tight_layout()
plt.show()

## 敏感核

单色频域波场是计算灵敏度核的关键，灵敏度核的计算将在接下来的练习中进行 ...

##### 练习

计算 5 Hz 频率域的敏感核 

\begin{equation} 
K(x,z,\omega) = {\cal{Re}}\biggl\{\frac{2 \omega^2}{Vp^3} \frac{\tilde{P}(x,z,\omega,\mathbf{x_{s}}) \tilde{G}(x,z,\omega,\mathbf{x_{r}})}{max(\tilde{P}(x,z,\omega,\mathbf{x_{s}}))}\biggr\} \notag
\end{equation}

对于均匀的走时成像 Marmousi-2 model. 

- 频率域的前向波场 $\tilde{P}(x,z,\omega,\mathbf{x_{s}})$ 对于震源在 $x_{s} = 2000.0\; m$ 和 $z_{s} = 40.0\; m$ 在笔记本的前几部分已经计算过了 (`p_hom_re`, `p_hom_im`,`p_fatt_re`, `p_fatt_im`). 你仅需要计算接收器的 Green's 函数 $\tilde{G}(x,z,\omega,\mathbf{x_{r}})$ 通过把震源放在接收器的位置 $x_{r} = 8000.0\; m$ and  $z_{r} = 40.0\; m$
- 计算敏感核 $K(x,z,\omega)$. Hint: In Python complex numbers are defined as `real_part + 1j*imag_part`. This can also be applied to `NumPy` arrays.
- 绘制、描述和解释均匀型和Marmousi-2型FATT模型的灵敏度核。在后续的FWI中，您期望在哪里进行模型更新? 

In [ ]:
# COMPUTE RECEIVER GREEN'S FUNCTION FOR HOMOGENEOUS MODEL HERE!
# -------------------------------------------------------------
%matplotlib notebook
op = 3  # define spatial FD operator (3-point) 

# Define time step
dt =  dx / (np.sqrt(2) * np.max(vp_hom))# time step (s)

g_hom_re, g_hom_im = FD_2D_acoustic_JIT(vp_hom,dt,dx,dz,f0,xsrc,zsrc,op,freq)

In [ ]:
%matplotlib notebook

# COMPUTE SENSITVITY KERNEL FOR HOMOGENEOUS MODEL HERE!

clip = 4e-18
extent = [0.0,xmax/1000,zmax/1000,0.0] # define model extension

fig = plt.figure(figsize=(7,3))  # define figure size

# Plot Vp-model
image = plt.imshow((vp_hom.T)/1000, cmap=plt.cm.gray, interpolation='nearest', 
                    extent=extent)    

# Plot Sensitivity Kernel
image1 = plt.imshow(K_hom.T, cmap="RdBu", alpha=.75, extent=extent, 
                      interpolation='nearest', vmin=-clip, vmax=clip)    
plt.title('Sensitivity Kernel (homogeneous model)')
plt.xlabel('x [km]')
plt.ylabel('z [km]')

plt.show()

In [ ]:
# COMPUTE RECEIVER GREEN'S FUNCTION FOR MARMOUSI-2 FATT MODEL HERE!
# -----------------------------------------------------------------
%matplotlib notebook
op = 3  # define spatial FD operator (3-point) 

# Define time step
dt =  dx / (np.sqrt(2) * np.max(vp_fatt))# time step (s)

g_fatt_re, g_fatt_im = FD_2D_acoustic_JIT(vp_fatt,dt,dx,dz,f0,xsrc,zsrc,op,freq)

In [ ]:
%matplotlib notebook

# COMPUTE SENSITVITY KERNEL FOR Marmousi-2 FATT MODEL HERE!
data = np.array(g_fatt_re + 1j * g_fatt_im)
K_hom = np.fft.fft(data)

clip = 8e-17
extent = [0.0,xmax/1000,zmax/1000,0.0] # define model extension

fig = plt.figure(figsize=(7,3))  # define figure size

# Plot Vp-model
image = plt.imshow((vp_fatt.T)/1000, cmap=plt.cm.gray, interpolation='nearest', 
                    extent=extent)    

# Plot Sensitivity Kernel
image1 = plt.imshow(K_hom.T, cmap="RdBu", alpha=.5, extent=extent, 
                      interpolation='nearest', vmin=-clip, vmax=clip)    
plt.title('Sensitivity Kernel (Marmousi-2 FATT model)')
plt.xlabel('x [km]')
plt.ylabel('z [km]')

plt.show()

## 我们学习到:

- 如何用离散傅里叶变换从时域波场计算单色频域波场
- 敏感核的计算